### Fitting models and materials database
- Materials database format:
  $$ [(ε_1, n_1, k_1), (ε_2, n_2, k_2), \cdots ] $$
  
- Fitted parameters database format:
  $$\{\{ϵ_∞\}, \{\{ω_i, γ_i\}, \cdots, \{ω_j, γ_j, f_j, g_j\}, \cdots\}\}$$
  
- Model includes Drude and critical-points (C-P) poles. If $g_j=0$, the C-P pole becomes Lorentz pole.
  $$\begin{align}
    ϵ(ω)
      & = ϵ_∞
        - \sum_i \frac{ω_i^2}{ω^2+iγ_iω}
        + \sum_j \frac{f_jω_j^2-ig_jω_jω}{ω_j^2-(ω+iγ_j)^2} \\
      & = ϵ_∞
        - \sum_i \frac{ω_i^2}{ω^2+iγ_iω}
        + \sum_j \frac12\left[i\frac{f_jω_j-g_j(γ_j+iω_j)}{(γ_j+iω_j)-iω} - i\frac{f_jω_j-g_j(γ_j-iω_j)}{(γ_j-iω_j)-iω}\right]
  \end{align}$$

In [ ]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt

MaterialsDB = {}
ParameterDB = {}

def ωλ(λ):
    return np.divide(1240, λ, where=λ!=0)

def nk(ω, model, param):
    ϵ, *p = param
    for i in range(model[0]):
        ωp, γp, *p = p
        ϵ -= ωp**2 / (ω**2+1j*γp*ω) # Drude pole
    for j in range(model[1]):
        ωp, γp, fp, gp, *p = p
        ϵ += (fp*ωp**2 - 1j*gp*ωp*ω) / (ωp**2 - (ω+1j*γp)**2) # C-P pole
    return ϵ**0.5

In [ ]:
def findFit(data, model, poles, constraints):
    x0 = [1.0]
    bounds = [(1.0, 10.0)]
    for i in [2, 4]: # 2: Drude, 4: C-P
        for j in range(model[i//2-1]):
            x0 += [1.0, 0.1, 1.0, 0.1][:i]
            if len(poles):
                try:
                    x0[len(x0)-i:len(x0)-i+len(poles[0])] = poles[0]
                except:
                    x0[len(x0)-i] = poles[0]
            bounds += [(0.0, 15.0), (0.0, None), (None, None), (None, None)][:i]
            poles = poles[1:]

    def f(x):
        vnk = np.vectorize(lambda ω: nk(ω, model, x))
        return sum(abs(vnk(data[:,0])-(data[:,1]+1j*data[:,2]))**2)

    return opt.minimize(f, x0, bounds=bounds, constraints=constraints)

In [ ]:
def plotFit(data, model, param, region):
    vnk = np.vectorize(lambda ω: nk(ω, model, param))
    λmin = 1/(1/region[0]+(1/region[0]-1/region[1])/32)
    λmax = 1/(1/region[1]-(1/region[0]-1/region[1])/32)
    ω = np.linspace(ωλ(λmax), ωλ(λmin), 201)
    n = vnk(ω)

    fig, ax = plt.subplots(figsize=(8,5))
    ax.plot(ωλ(ω), n.real, c='blue', label='n (fit)')
    ax.plot(ωλ(ω), n.imag, c='red' , label='k (fit)')
    ax.scatter(ωλ(data[:,0]), data[:,1], s=20, label='n (table)')
    ax.scatter(ωλ(data[:,0]), data[:,2], s=20, label='k (table)')
    ax.set_xscale('function', functions=(ωλ, ωλ))
    ax.set_xticks(1/np.linspace(1/(np.ceil(region[0]/50)*50), 1/(np.ceil(region[1]/region[0])*region[0]), 9))
    ax.set_xlim(λmin, λmax)
    ax.set_xlabel('Wavelength (nm)')
    ax.set_ylabel('Complex refractive index ($n$, $k$)')
    ax.invert_xaxis()
    bax = ax.secondary_xaxis('top', functions=(lambda λ: ωλ(λmax)+(ωλ(λmin)-ωλ(λmax))/(λmax-λmin)*(λ-λmin), lambda λ: λ))
    bax.set_xlabel('Photon Energy (eV)')
    bax.invert_xaxis()
    ax.grid()
    plt.show()

In [ ]:
def fitData(matter, region, model, poles=[], constraints=False):
    def σ(x): # Constraint: σ(x) > 0
        return sum(x[1+2*model[0]::4] * x[4+2*model[0]::4])

    i = max(0, np.argmin(MaterialsDB[matter][:,0] < ωλ(region[1]))-1)
    j = np.argmin(MaterialsDB[matter][:,0] < ωλ(region[0]))
    data = MaterialsDB[matter][i:j+1]

    fit = findFit(data, model, poles, {'type':'ineq', 'fun':σ} if constraints else None)

    x  = [fit.x[i:i+2] for i in range(1, 2*model[0], 2)]
    x += [fit.x[i:i+4] for i in range(1+2*model[0], 2*model[0]+4*model[1], 4)]
    output = f'{{{{{round(fit.x[0], 4)}}},{{{{{"},{".join(",".join(str(round(q, 4)) for q in p) for p in x)}}}}}}}'

    ParameterDB.setdefault('-'.join(map(str, region)), {})
    ParameterDB['-'.join(map(str, region))][matter.split('_')[0]] = output
    print(f'Converge: {fit.success}, StdDev: {round((fit.fun/len(data))**0.5, 3)}', end=', ')
    print(f'MaxParam: {round(max(abs(fit.x)), 4)}, DC-σ: {round(σ(fit.x), 4)}')
    print(f'matter {matter.split("_")[0]} = {output};')
    plotFit(data, model, fit.x, region)

In [ ]:
# Johnson and Christy, Phys. Rev. B 6, 4370 (1972)
MaterialsDB['Ag_JC']=np.array([(0.64,0.24,14.08),(0.77,0.15,11.85),(0.89,0.13,10.1),(1.02,0.09,8.828),(1.14,0.04,7.795),(1.26,0.04,6.992),(1.39,0.04,6.312),(1.51,0.04,5.727),(1.64,0.03,5.242),(1.76,0.04,4.838),(1.88,0.05,4.483),(2.01,0.06,4.152),(2.13,0.05,3.858),(2.26,0.06,3.586),(2.38,0.05,3.324),(2.5,0.05,3.093),(2.63,0.05,2.869),(2.75,0.04,2.657),(2.88,0.04,2.462),(3,0.05,2.275),(3.12,0.05,2.07),(3.25,0.05,1.864),(3.37,0.07,1.657),(3.5,0.1,1.419),(3.62,0.14,1.142),(3.74,0.17,0.829),(3.87,0.81,0.392),(3.99,1.13,0.616),(4.12,1.34,0.964),(4.24,1.39,1.161),(4.36,1.41,1.264),(4.49,1.41,1.331),(4.61,1.38,1.372),(4.74,1.35,1.387),(4.86,1.33,1.393),(4.98,1.31,1.389)])
MaterialsDB['Au_JC']=np.array([(0.64,0.92,13.78),(0.77,0.56,11.21),(0.89,0.43,9.519),(1.02,0.35,8.145),(1.14,0.27,7.15),(1.26,0.22,6.35),(1.39,0.17,5.663),(1.51,0.16,5.083),(1.64,0.14,4.542),(1.76,0.13,4.103),(1.88,0.14,3.697),(2.01,0.21,3.272),(2.13,0.29,2.863),(2.26,0.43,2.455),(2.38,0.62,2.081),(2.5,1.04,1.833),(2.63,1.31,1.849),(2.75,1.38,1.914),(2.88,1.45,1.948),(3,1.46,1.958),(3.12,1.47,1.952),(3.25,1.46,1.933),(3.37,1.48,1.895),(3.5,1.5,1.866),(3.62,1.48,1.871),(3.74,1.48,1.883),(3.87,1.54,1.898),(3.99,1.53,1.893),(4.12,1.53,1.889),(4.24,1.49,1.878),(4.36,1.47,1.869),(4.49,1.43,1.847),(4.61,1.38,1.803),(4.74,1.35,1.749),(4.86,1.33,1.688),(4.98,1.33,1.631)])
MaterialsDB['Cu_JC']=np.array([(0.64,1.09,13.43),(0.77,0.76,11.12),(0.89,0.6,9.439),(1.02,0.48,8.245),(1.14,0.36,7.217),(1.26,0.32,6.421),(1.39,0.3,5.768),(1.51,0.26,5.18),(1.64,0.24,4.665),(1.76,0.21,4.205),(1.88,0.22,3.747),(2.01,0.3,3.205),(2.13,0.7,2.704),(2.26,1.02,2.577),(2.38,1.18,2.608),(2.5,1.22,2.564),(2.63,1.25,2.483),(2.75,1.24,2.397),(2.88,1.25,2.305),(3,1.28,2.207),(3.12,1.32,2.116),(3.25,1.33,2.045),(3.37,1.36,1.975),(3.5,1.37,1.916),(3.62,1.36,1.864),(3.74,1.34,1.821),(3.87,1.38,1.783),(3.99,1.38,1.729),(4.12,1.4,1.679),(4.24,1.42,1.633),(4.36,1.45,1.633),(4.49,1.46,1.646),(4.61,1.45,1.668),(4.74,1.41,1.691),(4.86,1.41,1.741),(4.98,1.37,1.783)])

# Handbook of Chemistry & Physics (2006)
MaterialsDB['Ag']=np.array([(0.5,0.67,18.32),(1.0,0.28,9.03),(1.5,0.27,5.79),(2.0,0.27,4.18),(2.5,0.24,3.09),(3.0,0.23,2.27),(3.25,0.23,1.86),(3.5,0.21,1.42),(3.6,0.23,1.13),(3.7,0.3,0.77),(3.77,0.53,0.40),(3.8,0.73,0.30),(3.9,1.3,0.36),(4.0,1.61,0.60),(4.1,1.73,0.85),(4.2,1.75,1.06),(4.3,1.73,1.13),(4.5,1.69,1.28),(4.75,1.61,1.34),(5.0,1.55,1.36)])
MaterialsDB['Au']=np.array([(0.5,0.39,16.61),(0.6,0.28,13.78),(0.7,0.22,11.75),(0.8,0.18,10.21),(0.9,0.15,9.01),(1.0,0.13,8.03),(1.2,0.1,6.54),(1.4,0.08,5.44),(1.6,0.08,4.56),(1.8,0.09,3.82),(2.0,0.13,3.16),(2.1,0.18,2.84),(2.2,0.24,2.54),(2.4,0.5,1.86),(2.5,0.82,1.59),(2.6,1.24,1.54),(2.7,1.43,1.72),(2.8,1.46,1.77),(2.9,1.5,1.79),(3.0,1.54,1.80),(3.1,1.54,1.81),(3.2,1.54,1.80),(3.3,1.55,1.78),(3.4,1.56,1.76),(3.5,1.58,1.73),(3.6,1.62,1.73),(3.7,1.64,1.75),(3.8,1.63,1.79),(3.9,1.59,1.81),(4.0,1.55,1.81),(4.1,1.51,1.79),(4.2,1.48,1.78),(4.3,1.45,1.77),(4.4,1.41,1.76),(4.5,1.35,1.74),(4.6,1.3,1.69),(4.7,1.27,1.64),(4.8,1.25,1.59),(4.9,1.23,1.54),(5.0,1.22,1.49),(5.2,1.21,1.40),(5.4,1.21,1.33),(5.6,1.21,1.27),(5.8,1.21,1.20),(6.0,1.22,1.14),(6.2,1.24,1.09),(6.4,1.25,1.05),(6.6,1.27,1.01),(6.8,1.3,0.97),(7.0,1.34,0.95)])
MaterialsDB['Cu']=np.array([(0.5,1.71,17.63),(1.0,0.44,8.48),(1.5,0.26,5.26),(1.7,0.22,4.43),(1.75,0.21,4.25),(1.8,0.21,4.04),(1.85,0.22,3.85),(1.9,0.21,3.67),(2.0,0.27,3.24),(2.1,0.47,2.81),(2.2,0.83,2.60),(2.3,1.04,2.59),(2.4,1.12,2.60),(2.6,1.15,2.50),(2.8,1.17,2.36),(3.0,1.18,2.21),(3.2,1.23,2.07),(3.4,1.27,1.95),(3.6,1.31,1.87),(3.8,1.34,1.81),(4.0,1.34,1.72),(4.2,1.42,1.64),(4.4,1.49,1.64),(4.6,1.52,1.67),(4.8,1.53,1.71),(5.0,1.47,1.78),(5.2,1.38,1.80),(5.4,1.28,1.78),(5.6,1.18,1.74),(5.8,1.1,1.67),(6.0,1.04,1.59),(6.5,0.96,1.37),(7.0,0.97,1.20)])
MaterialsDB['Al']=np.array([(0.6,2.273,21.403),(0.7,1.77,18.328),(0.8,1.444,15.955),(0.9,1.264,14.021),(1.,1.212,12.464),(1.1,1.201,11.181),(1.2,1.26,10.010),(1.3,1.468,8.949),(1.4,2.237,8.212),(1.5,2.745,8.309),(1.6,2.625,8.597),(1.7,2.143,8.573),(1.8,1.741,8.205),(1.9,1.488,7.821),(2.,1.304,7.479),(2.2,1.018,6.846),(2.4,0.826,6.283),(2.6,0.695,5.800),(2.8,0.598,5.385),(3.,0.523,5.024),(3.2,0.46,4.708),(3.4,0.407,4.426),(3.6,0.363,4.174),(3.8,0.326,3.946),(4.,0.294,3.740),(4.2,0.267,3.552),(4.4,0.244,3.380),(4.6,0.223,3.222),(4.8,0.205,3.076),(5.,0.19,2.942)])
MaterialsDB['Cr']=np.array([(0.54,3.92,7.06),(0.66,3.96,5.95),(0.78,4.13,5.03),(0.9,4.43,4.60),(1.,4.47,4.43),(1.12,4.53,4.31),(1.24,4.5,4.28),(1.36,4.42,4.30),(1.46,4.31,4.32),(1.77,3.84,4.37),(2.,3.48,4.36),(2.2,3.18,4.41),(2.4,2.75,4.46),(2.6,2.22,4.36),(2.8,1.8,4.06),(3.,1.54,3.71),(3.2,1.44,3.40),(3.4,1.39,3.24),(3.6,1.26,3.12),(3.8,1.12,2.95),(4.,1.02,2.76),(4.2,0.94,2.58),(4.4,0.9,2.42),(4.5,0.89,2.35),(4.6,0.88,2.28),(4.7,0.86,2.21),(4.8,0.86,2.13),(4.9,0.86,2.07),(5.,0.85,2.01)])
MaterialsDB['Ni']=np.array([(0.6,3.84,8.35),(0.7,3.59,7.48),(0.8,3.38,6.82),(0.9,3.18,6.23),(1.,3.06,5.74),(1.1,2.97,5.38),(1.2,2.85,5.10),(1.3,2.74,4.85),(1.4,2.65,4.63),(1.5,2.53,4.47),(1.6,2.43,4.31),(1.7,2.28,4.18),(1.8,2.14,4.01),(1.9,2.02,3.82),(2.,1.92,3.65),(2.1,1.85,3.48),(2.2,1.8,3.33),(2.3,1.75,3.19),(2.4,1.71,3.06),(2.5,1.67,2.93),(2.6,1.65,2.81),(2.7,1.64,2.71),(2.8,1.63,2.61),(2.9,1.62,2.52),(3.,1.61,2.44),(3.1,1.61,2.36),(3.2,1.61,2.30),(3.3,1.61,2.23),(3.4,1.62,2.17),(3.5,1.63,2.11),(3.6,1.64,2.07),(3.7,1.66,2.02),(3.8,1.69,1.99),(3.9,1.72,1.98),(4.,1.73,1.98),(4.2,1.74,2.01),(4.4,1.71,2.06),(4.6,1.63,2.09),(4.8,1.53,2.11),(5.,1.4,2.10)])
MaterialsDB['Pd']=np.array([(0.6,3.8,9.96),(0.72,3.51,8.70),(0.8,3.35,8.06),(1.0,2.99,6.89),(1.1,2.81,6.46),(1.2,2.65,6.10),(1.3,2.5,5.78),(1.4,2.34,5.50),(1.5,2.17,5.22),(1.6,2.08,4.95),(1.7,2.0,4.72),(1.8,1.92,4.54),(1.9,1.82,4.35),(2.0,1.75,4.18),(2.1,1.67,4.03),(2.2,1.6,3.88),(2.3,1.53,3.75),(2.4,1.47,3.61),(2.5,1.41,3.48),(2.6,1.37,3.36),(2.7,1.32,3.25),(2.8,1.29,3.13),(2.9,1.26,3.03),(3.0,1.23,2.94),(3.1,1.2,2.85),(3.2,1.17,2.77),(3.3,1.14,2.68),(3.4,1.12,2.60),(3.5,1.1,2.52),(3.6,1.08,2.45),(3.7,1.07,2.38),(3.8,1.06,2.31),(3.9,1.05,2.25),(4.0,1.03,2.19),(4.2,1.04,2.09),(4.4,1.03,2.01),(4.6,1.03,1.94),(4.8,1.01,1.90),(5.0,0.96,1.86)])
MaterialsDB['Pt']=np.array([(0.5,3.91,7.71),(0.55,4.58,7.14),(0.6,5.13,6.75),(0.65,5.52,6.66),(0.7,5.71,6.83),(0.75,5.57,7.02),(0.8,5.31,7.04),(0.85,5.05,6.98),(0.9,4.77,6.91),(0.95,4.5,6.77),(1.,4.25,6.62),(1.1,3.86,6.24),(1.2,3.55,5.92),(1.3,3.29,5.61),(1.4,3.1,5.32),(1.5,2.92,5.07),(1.6,2.76,4.84),(1.7,2.63,4.64),(1.8,2.51,4.43),(1.9,2.38,4.26),(2.,2.3,4.07),(2.1,2.23,3.92),(2.2,2.17,3.77),(2.3,2.1,3.67),(2.4,2.03,3.54),(2.5,1.96,3.42),(2.6,1.91,3.30),(2.7,1.87,3.20),(2.8,1.83,3.10),(2.9,1.79,3.01),(3.,1.75,2.92),(3.2,1.68,2.76),(3.4,1.63,2.62),(3.6,1.58,2.48),(3.8,1.53,2.37),(4.,1.49,2.25),(4.2,1.45,2.14),(4.4,1.43,2.04),(4.6,1.39,1.95),(4.8,1.38,1.85),(5.,1.36,1.76)])
MaterialsDB['Ti']=np.array([(0.6,4.71,3.77),(0.7,4.38,3.89),(0.8,4.04,3.82),(0.9,3.8,3.65),(1.,3.62,3.52),(1.1,3.47,3.40),(1.2,3.35,3.30),(1.3,3.28,3.25),(1.4,3.17,3.28),(1.5,2.98,3.32),(1.6,2.74,3.30),(1.7,2.54,3.23),(1.8,2.36,3.11),(1.9,2.22,2.99),(2.,2.11,2.88),(2.1,2.01,2.77),(2.2,1.92,2.67),(2.3,1.86,2.56),(2.4,1.81,2.47),(2.5,1.78,2.39),(2.6,1.75,2.34),(2.7,1.71,2.29),(2.8,1.68,2.25),(2.9,1.63,2.21),(3.,1.59,2.17),(3.1,1.55,2.15),(3.2,1.5,2.12),(3.3,1.44,2.09),(3.4,1.37,2.06),(3.5,1.3,2.01),(3.6,1.24,1.96),(3.7,1.17,1.90),(3.8,1.11,1.83),(3.85,1.08,1.78),(3.9,1.06,1.73),(4.,1.04,1.62),(4.2,1.05,1.45),(4.4,1.13,1.33),(4.6,1.17,1.29),(4.8,1.21,1.23),(5.,1.24,1.21),(5.2,1.27,1.20)])
MaterialsDB['Ge']=np.array([(0.6,4.104,6.58e-07),(0.7,4.18,1.27e-04),(0.8,4.275,5.67e-03),(0.9,4.285,7.45e-02),(1.,4.325,8.09e-02),(1.1,4.385,0.103),(1.2,4.42,0.123),(1.3,4.495,0.167),(1.4,4.56,0.190),(1.5,4.635,0.298),(1.6,4.763,0.345),(1.7,4.897,0.401),(1.8,5.067,0.500),(1.9,5.38,0.540),(2.,5.588,0.933),(2.1,5.748,1.634),(2.2,5.283,2.049),(2.3,5.062,2.318),(2.4,4.61,2.455),(2.5,4.34,2.384),(2.6,4.18,2.309),(2.7,4.082,2.240),(2.8,4.035,2.181),(2.9,4.037,2.140),(3.,4.082,2.145),(3.1,4.141,2.215),(3.2,4.157,2.340),(3.3,4.128,2.469),(3.4,4.07,2.579),(3.5,4.02,2.667),(3.6,3.985,2.759),(3.7,3.958,2.863),(3.8,3.936,2.986),(3.9,3.92,3.137),(4.,3.905,3.336),(4.1,3.869,3.614),(4.2,3.745,4.009),(4.3,3.338,4.507),(4.4,2.516,4.669),(4.5,1.953,4.297),(4.6,1.72,3.960),(4.7,1.586,3.709),(4.8,1.498,3.509),(4.9,1.435,3.342),(5.,1.394,3.197),(5.1,1.37,3.073),(5.2,1.364,2.973),(5.3,1.371,2.897),(5.4,1.383,2.854),(5.5,1.38,2.842),(5.6,1.36,2.846),(5.7,1.293,2.163),(5.8,1.209,2.873),(5.9,1.108,2.813),(6.,1.30,2.340)])
MaterialsDB['Si']=np.array([(0.6199,3.449,0),(0.8093,3.4784,0),(1.033,3.5193,0),(1.1,3.5341,0.000013),(1.2,3.568825,0.00018),(1.3,3.60355,0.00226),(1.4,3.638275,0.00775),(1.5,3.673,0.005),(1.6,3.714,0.008),(1.7,3.752,0.01),(1.8,3.796,0.013),(1.9,3.847,0.016),(2.,3.906,0.022),(2.1,3.969,0.030),(2.2,4.042,0.032),(2.3,4.123,0.048),(2.4,4.215,0.060),(2.5,4.32,0.073),(2.6,4.442,0.090),(2.7,4.583,0.130),(2.8,4.753,0.163),(2.9,4.961,0.203),(3.,5.222,0.269),(3.1,5.57,0.387),(3.2,6.062,0.630),(3.3,6.709,1.321),(3.4,6.522,2.705),(3.5,5.61,3.014),(3.6,5.296,2.987),(3.7,5.156,3.058),(3.8,5.065,3.182),(3.9,5.016,3.346),(4.,5.01,3.587),(4.1,5.02,3.979),(4.2,4.888,4.639),(4.3,4.086,5.395),(4.4,3.12,5.344),(4.5,2.451,5.082),(4.6,1.988,4.678),(4.7,1.764,4.278),(4.8,1.658,3.979),(4.9,1.597,3.749),(5.,1.57,3.565),(5.1,1.571,3.429),(5.2,1.589,3.354),(5.3,1.579,3.353),(5.4,1.471,3.366),(5.5,1.34,3.302),(5.6,1.247,3.206),(5.7,1.18,3.112),(5.8,1.133,3.045),(5.9,1.083,2.982),(6.,1.01,2.909)])
MaterialsDB['aSi']=np.array([(0.6,3.44,0.0),(0.7,3.45,0.0),(0.8,3.48,0.0),(0.9,3.5,0.0),(1.0,3.54,0.0),(1.1,3.57,0.0),(1.2,3.61,0.0),(1.3,3.68,0.0),(1.4,3.77,0.0401),(1.5,3.86,0.0812),(1.6,3.93,0.136),(1.7,4.01,0.199),(1.8,4.09,0.271),(1.9,4.17,0.363),(2.0,4.23,0.461),(2.2,4.36,0.69),(2.4,4.46,0.969),(2.5,4.47,1.12),(2.6,4.49,1.28),(2.8,4.47,1.64),(3.0,4.38,2.02),(3.2,4.17,2.38),(3.4,3.9,2.66),(3.5,3.73,2.79),(3.6,3.55,2.88),(3.8,3.21,3.0),(4.0,2.87,3.06),(4.2,2.56,3.04),(4.4,2.3,2.99),(4.6,2.07,2.93),(4.8,1.86,2.85),(5.0,1.69,2.76),(5.5,1.35,2.51),(6.0,1.11,2.28),(6.5,0.951,2.07),(7.0,0.832,1.89),(7.5,0.735,1.74),(8.0,0.66,1.6),(8.5,0.597,1.47),(9.0,0.543,1.35),(9.5,0.497,1.24),(10.0,0.459,1.14),(10.5,0.423,1.04),(11.0,0.392,0.946),(11.5,0.363,0.847),(12.0,0.327,0.726)])

# S. Ninomiya and S. Adachi., J. Appl. Phys. 78, 1183-1190 (1995)
MaterialsDB['oCdS']=np.array([(1.2,2.1128,0.02387),(1.209,2.1139,0.02401),(1.218,2.115,0.02415),(1.227,2.1161,0.02429),(1.236,2.1172,0.02443),(1.245,2.1184,0.02457),(1.254,2.1195,0.02471),(1.263,2.1207,0.02486),(1.272,2.1219,0.025),(1.281,2.1231,0.02515),(1.29,2.1243,0.0253),(1.299,2.1255,0.02545),(1.308,2.1267,0.0256),(1.317,2.128,0.02575),(1.326,2.1292,0.02591),(1.335,2.1305,0.02606),(1.344,2.1317,0.02622),(1.353,2.133,0.02638),(1.362,2.1343,0.02654),(1.371,2.1356,0.0267),(1.38,2.1369,0.02686),(1.389,2.1383,0.02702),(1.398,2.1396,0.02719),(1.407,2.141,0.02736),(1.416,2.1423,0.02752),(1.425,2.1437,0.02769),(1.434,2.1451,0.02787),(1.443,2.1465,0.02804),(1.452,2.1479,0.02821),(1.461,2.1494,0.02839),(1.47,2.1508,0.02857),(1.479,2.1523,0.02875),(1.488,2.1538,0.02893),(1.497,2.1553,0.02912),(1.506,2.1568,0.02931),(1.515,2.1583,0.02949),(1.524,2.1599,0.02969),(1.533,2.1614,0.02988),(1.542,2.163,0.03007),(1.551,2.1646,0.03027),(1.56,2.1662,0.03047),(1.569,2.1678,0.03067),(1.578,2.1695,0.03088),(1.587,2.1711,0.03109),(1.596,2.1728,0.03129),(1.606,2.1745,0.03151),(1.615,2.1762,0.03172),(1.624,2.178,0.03194),(1.633,2.1797,0.03216),(1.642,2.1815,0.03238),(1.651,2.1833,0.03261),(1.66,2.1851,0.03284),(1.669,2.187,0.03307),(1.678,2.1888,0.03331),(1.687,2.1907,0.03355),(1.696,2.1926,0.03379),(1.705,2.1946,0.03404),(1.714,2.1965,0.03429),(1.723,2.1985,0.03454),(1.732,2.2005,0.0348),(1.741,2.2025,0.03506),(1.75,2.2046,0.03533),(1.759,2.2066,0.0356),(1.768,2.2088,0.03588),(1.777,2.2109,0.03616),(1.786,2.2131,0.03644),(1.795,2.2152,0.03673),(1.804,2.2175,0.03703),(1.813,2.2197,0.03733),(1.822,2.222,0.03763),(1.831,2.2243,0.03795),(1.84,2.2267,0.03826),(1.849,2.229,0.03859),(1.858,2.2315,0.03892),(1.867,2.2339,0.03926),(1.876,2.2364,0.03961),(1.885,2.2389,0.03996),(1.894,2.2415,0.04032),(1.903,2.2441,0.04069),(1.912,2.2468,0.04107),(1.921,2.2495,0.04146),(1.93,2.2522,0.04186),(1.939,2.255,0.04226),(1.948,2.2578,0.04268),(1.957,2.2607,0.04311),(1.966,2.2636,0.04356),(1.975,2.2666,0.04401),(1.984,2.2697,0.04448),(1.993,2.2728,0.04496),(2.002,2.2759,0.04546),(2.011,2.2791,0.04597),(2.02,2.2824,0.04651),(2.029,2.2858,0.04705),(2.038,2.2892,0.04762),(2.047,2.2927,0.04821),(2.056,2.2962,0.04882),(2.065,2.2999,0.04946),(2.075,2.3036,0.05012),(2.084,2.3074,0.05081),(2.093,2.3113,0.05153),(2.102,2.3154,0.05228),(2.111,2.3195,0.05307),(2.12,2.3237,0.05389),(2.129,2.328,0.05476),(2.138,2.3324,0.05567),(2.147,2.337,0.05663),(2.156,2.3417,0.05765),(2.165,2.3465,0.05873),(2.174,2.3515,0.05987),(2.183,2.3567,0.06109),(2.192,2.362,0.06239),(2.201,2.3675,0.06378),(2.21,2.3732,0.06527),(2.219,2.379,0.06687),(2.228,2.3852,0.06861),(2.237,2.3915,0.0705),(2.246,2.3981,0.07255),(2.255,2.405,0.07479),(2.264,2.4122,0.07726),(2.273,2.4197,0.07998),(2.282,2.4275,0.083),(2.291,2.4357,0.08637),(2.3,2.4444,0.09015),(2.309,2.4534,0.09441),(2.318,2.4629,0.09926),(2.327,2.473,0.10481),(2.336,2.4835,0.1112),(2.345,2.4945,0.11862),(2.354,2.506,0.1273),(2.363,2.518,0.13753),(2.372,2.5302,0.14966),(2.381,2.5425,0.16411),(2.39,2.5544,0.18136),(2.399,2.5651,0.20189),(2.408,2.5736,0.22607),(2.417,2.5783,0.25387),(2.426,2.5773,0.2845),(2.435,2.5687,0.31595),(2.444,2.5518,0.34492),(2.453,2.5278,0.36758),(2.462,2.5001,0.38118),(2.471,2.4734,0.38552),(2.48,2.4514,0.38305),(2.489,2.4352,0.37746),(2.498,2.4239,0.37186),(2.507,2.4153,0.3676),(2.516,2.4074,0.36426),(2.525,2.3995,0.36051),(2.535,2.3924,0.3553),(2.544,2.3872,0.34868),(2.553,2.3846,0.34158),(2.562,2.3847,0.33512),(2.571,2.3867,0.33006),(2.58,2.3899,0.32667),(2.589,2.3937,0.32492),(2.598,2.3977,0.3246),(2.607,2.4014,0.32545),(2.616,2.4049,0.32722),(2.625,2.408,0.3297),(2.634,2.4106,0.33272),(2.643,2.4129,0.33613),(2.652,2.4148,0.33984),(2.661,2.4163,0.34375),(2.67,2.4175,0.3478),(2.679,2.4184,0.35194),(2.688,2.419,0.35613),(2.697,2.4193,0.36035),(2.706,2.4195,0.36456),(2.715,2.4194,0.36876),(2.724,2.4192,0.37291),(2.733,2.4188,0.37703),(2.742,2.4183,0.38109),(2.751,2.4177,0.38509),(2.76,2.4169,0.38903),(2.769,2.4161,0.39289),(2.778,2.4151,0.39669),(2.787,2.4141,0.40042),(2.796,2.413,0.40408),(2.805,2.4118,0.40766),(2.814,2.4106,0.41117),(2.823,2.4093,0.4146),(2.832,2.408,0.41797),(2.841,2.4067,0.42126),(2.85,2.4053,0.42449),(2.859,2.4039,0.42764),(2.868,2.4024,0.43073),(2.877,2.401,0.43375),(2.886,2.3995,0.43671),(2.895,2.398,0.4396),(2.904,2.3965,0.44243),(2.913,2.395,0.44519),(2.922,2.3935,0.4479),(2.931,2.392,0.45055),(2.94,2.3905,0.45314),(2.949,2.389,0.45568),(2.958,2.3875,0.45816),(2.967,2.386,0.46059),(2.976,2.3845,0.46297),(2.985,2.383,0.4653),(2.994,2.3815,0.46757),(3.004,2.38,0.4698),(3.013,2.3786,0.47199),(3.022,2.3771,0.47413),(3.031,2.3757,0.47622),(3.04,2.3743,0.47827),(3.049,2.3729,0.48028),(3.058,2.3715,0.48225),(3.067,2.3702,0.48418),(3.076,2.3688,0.48607),(3.085,2.3675,0.48792),(3.094,2.3662,0.48974),(3.103,2.3649,0.49152),(3.112,2.3636,0.49326),(3.121,2.3624,0.49498),(3.13,2.3612,0.49666),(3.139,2.36,0.49831),(3.148,2.3588,0.49992),(3.157,2.3577,0.50151),(3.166,2.3566,0.50307),(3.175,2.3555,0.5046),(3.184,2.3544,0.5061),(3.193,2.3534,0.50758),(3.202,2.3523,0.50903),(3.211,2.3513,0.51046),(3.22,2.3504,0.51186),(3.229,2.3494,0.51324),(3.238,2.3485,0.5146),(3.247,2.3477,0.51594),(3.256,2.3468,0.51725),(3.265,2.346,0.51855),(3.274,2.3452,0.51982),(3.283,2.3444,0.52108),(3.292,2.3437,0.52232),(3.301,2.343,0.52354),(3.31,2.3423,0.52474),(3.319,2.3417,0.52593),(3.328,2.3411,0.52711),(3.337,2.3405,0.52826),(3.346,2.3399,0.52941),(3.355,2.3394,0.53054),(3.364,2.3389,0.53166),(3.373,2.3385,0.53277),(3.382,2.3381,0.53387),(3.391,2.3377,0.53495),(3.4,2.3373,0.53603),(3.409,2.337,0.5371),(3.418,2.3367,0.53816),(3.427,2.3364,0.53921),(3.436,2.3362,0.54025),(3.445,2.336,0.54129),(3.454,2.3359,0.54232),(3.463,2.3358,0.54334),(3.473,2.3357,0.54436),(3.481,2.3356,0.54538),(3.49,2.3356,0.54639),(3.5,2.3356,0.5474),(3.509,2.3357,0.54841),(3.518,2.3358,0.54942),(3.527,2.3359,0.55043),(3.536,2.3361,0.55143),(3.545,2.3363,0.55244),(3.554,2.3365,0.55345),(3.563,2.3368,0.55446),(3.572,2.3371,0.55547),(3.581,2.3375,0.55648),(3.59,2.3378,0.5575),(3.599,2.3383,0.55853),(3.608,2.3387,0.55956),(3.617,2.3392,0.56059),(3.626,2.3398,0.56163),(3.635,2.3404,0.56268),(3.644,2.341,0.56374),(3.653,2.3416,0.56481),(3.662,2.3423,0.56588),(3.671,2.3431,0.56697),(3.68,2.3439,0.56807),(3.689,2.3447,0.56918),(3.698,2.3455,0.5703),(3.707,2.3465,0.57144),(3.716,2.3474,0.57259),(3.725,2.3484,0.57375),(3.734,2.3494,0.57494),(3.743,2.3505,0.57614),(3.752,2.3516,0.57736),(3.761,2.3528,0.57859),(3.77,2.3539,0.57985),(3.779,2.3552,0.58113),(3.788,2.3565,0.58243),(3.797,2.3578,0.58375),(3.806,2.3592,0.5851),(3.815,2.3606,0.58647),(3.824,2.3621,0.58786),(3.833,2.3636,0.58929),(3.842,2.3651,0.59074),(3.851,2.3667,0.59222),(3.86,2.3684,0.59374),(3.869,2.37,0.59528),(3.878,2.3718,0.59686),(3.887,2.3736,0.59847),(3.896,2.3754,0.60011),(3.905,2.3773,0.6018),(3.914,2.3792,0.60352),(3.923,2.3811,0.60528),(3.933,2.3832,0.60708),(3.942,2.3852,0.60893),(3.951,2.3873,0.61081),(3.96,2.3895,0.61275),(3.969,2.3917,0.61473),(3.978,2.3939,0.61676),(3.987,2.3962,0.61884),(3.996,2.3986,0.62097),(4.005,2.401,0.62316),(4.014,2.4034,0.6254),(4.023,2.4059,0.6277),(4.032,2.4084,0.63006),(4.041,2.411,0.63248),(4.05,2.4137,0.63496),(4.059,2.4163,0.63751),(4.068,2.4191,0.64013),(4.077,2.4218,0.64282),(4.086,2.4247,0.64558),(4.095,2.4275,0.64841),(4.104,2.4304,0.65132),(4.113,2.4334,0.65432),(4.122,2.4364,0.65739),(4.131,2.4395,0.66055),(4.14,2.4425,0.66379),(4.149,2.4457,0.66713),(4.158,2.4489,0.67055),(4.167,2.4521,0.67408),(4.176,2.4553,0.6777),(4.185,2.4586,0.68142),(4.194,2.462,0.68525),(4.203,2.4653,0.68918),(4.212,2.4688,0.69323),(4.221,2.4722,0.69739),(4.23,2.4757,0.70167),(4.239,2.4792,0.70607),(4.248,2.4827,0.7106),(4.257,2.4863,0.71525),(4.266,2.4899,0.72004),(4.275,2.4935,0.72496),(4.284,2.4971,0.73002),(4.293,2.5007,0.73523),(4.302,2.5044,0.74058),(4.311,2.508,0.74608),(4.32,2.5117,0.75174),(4.329,2.5154,0.75756),(4.338,2.519,0.76354),(4.347,2.5227,0.76969),(4.356,2.5264,0.77601),(4.365,2.53,0.7825),(4.374,2.5336,0.78917),(4.383,2.5372,0.79603),(4.392,2.5407,0.80307),(4.402,2.5442,0.8103),(4.41,2.5477,0.81773),(4.42,2.5511,0.82536),(4.429,2.5544,0.83318),(4.438,2.5577,0.84121),(4.446,2.5609,0.84944),(4.456,2.564,0.85789),(4.465,2.567,0.86654),(4.474,2.5699,0.87541),(4.483,2.5727,0.88449),(4.492,2.5754,0.89379),(4.501,2.5779,0.9033),(4.51,2.5803,0.91303),(4.519,2.5825,0.92298),(4.528,2.5845,0.93314),(4.537,2.5864,0.94351),(4.546,2.588,0.95409),(4.555,2.5894,0.96488),(4.564,2.5906,0.97587),(4.573,2.5916,0.98705),(4.582,2.5923,0.99843),(4.591,2.5927,1.01),(4.6,2.5929,1.0217),(4.609,2.5927,1.0336),(4.618,2.5922,1.0456),(4.627,2.5915,1.0578),(4.636,2.5903,1.0701),(4.645,2.5888,1.0826),(4.654,2.587,1.0951),(4.663,2.5848,1.1077),(4.672,2.5821,1.1203),(4.681,2.5791,1.133),(4.69,2.5757,1.1457),(4.699,2.5719,1.1584),(4.708,2.5677,1.1711),(4.717,2.563,1.1837),(4.726,2.5579,1.1962),(4.735,2.5524,1.2086),(4.744,2.5465,1.2209),(4.753,2.5402,1.2331),(4.762,2.5334,1.245),(4.771,2.5263,1.2567),(4.78,2.5188,1.2682),(4.789,2.5109,1.2795),(4.798,2.5027,1.2904),(4.807,2.4941,1.301),(4.816,2.4852,1.3113),(4.825,2.4761,1.3213),(4.834,2.4666,1.3308),(4.843,2.4569,1.34),(4.852,2.447,1.3488),(4.861,2.437,1.3571),(4.87,2.4267,1.365),(4.879,2.4164,1.3725),(4.889,2.406,1.3796),(4.898,2.3955,1.3862),(4.907,2.385,1.3923),(4.916,2.3745,1.398),(4.925,2.3641,1.4033),(4.934,2.3537,1.4082),(4.943,2.3434,1.4127),(4.952,2.3333,1.4168),(4.961,2.3233,1.4205),(4.97,2.3136,1.4238),(4.979,2.304,1.4269),(4.988,2.2947,1.4296),(4.997,2.2856,1.432),(5.006,2.2768,1.4342),(5.015,2.2683,1.4362),(5.024,2.26,1.4379),(5.033,2.2521,1.4395),(5.042,2.2445,1.441),(5.051,2.2372,1.4424),(5.06,2.2303,1.4436),(5.069,2.2236,1.4449),(5.078,2.2173,1.4461),(5.087,2.2113,1.4473),(5.096,2.2056,1.4486),(5.105,2.2001,1.45),(5.114,2.195,1.4515),(5.123,2.1901,1.4531),(5.132,2.1855,1.4548),(5.141,2.1811,1.4568),(5.15,2.1769,1.4589),(5.159,2.1729,1.4612),(5.168,2.169,1.4638),(5.177,2.1653,1.4667),(5.186,2.1617,1.4698),(5.195,2.1582,1.4732),(5.204,2.1547,1.4769),(5.213,2.1512,1.4809),(5.222,2.1477,1.4851),(5.231,2.1442,1.4897),(5.24,2.1406,1.4946),(5.249,2.1368,1.4998),(5.258,2.133,1.5053),(5.267,2.1289,1.511),(5.276,2.1246,1.5171),(5.285,2.1202,1.5233),(5.294,2.1154,1.5298),(5.303,2.1103,1.5366),(5.313,2.105,1.5435),(5.321,2.0992,1.5506),(5.331,2.0931,1.5578),(5.339,2.0867,1.5651),(5.348,2.0798,1.5724),(5.357,2.0725,1.5799),(5.367,2.0648,1.5873),(5.376,2.0566,1.5946),(5.384,2.048,1.6019),(5.394,2.039,1.6091),(5.403,2.0296,1.6161),(5.412,2.0197,1.6229),(5.421,2.0094,1.6295),(5.43,1.9987,1.6358),(5.439,1.9877,1.6418),(5.448,1.9763,1.6475),(5.457,1.9645,1.6528),(5.466,1.9525,1.6576),(5.475,1.9402,1.6621),(5.484,1.9276,1.6661),(5.493,1.9149,1.6696),(5.502,1.902,1.6727),(5.511,1.889,1.6752),(5.52,1.8759,1.6773),(5.529,1.8627,1.6788),(5.538,1.8496,1.6798),(5.547,1.8365,1.6803),(5.556,1.8234,1.6803),(5.565,1.8105,1.6798),(5.574,1.7977,1.6788),(5.583,1.7851,1.6773),(5.592,1.7727,1.6754),(5.601,1.7606,1.673),(5.61,1.7487,1.6702),(5.619,1.7372,1.667),(5.628,1.726,1.6635),(5.637,1.7151,1.6596),(5.646,1.7046,1.6555),(5.655,1.6944,1.651),(5.664,1.6847,1.6463),(5.673,1.6754,1.6414),(5.682,1.6665,1.6362),(5.691,1.658,1.6309),(5.7,1.65,1.6255)])
MaterialsDB['eCdS']=np.array([(1.2,2.13,0.02387),(1.209,2.1312,0.02401),(1.218,2.1324,0.02415),(1.227,2.1336,0.02429),(1.236,2.1349,0.02443),(1.245,2.1361,0.02457),(1.254,2.1373,0.02471),(1.263,2.1386,0.02486),(1.272,2.1398,0.025),(1.281,2.1411,0.02515),(1.29,2.1424,0.0253),(1.299,2.1437,0.02545),(1.308,2.145,0.0256),(1.317,2.1463,0.02575),(1.326,2.1476,0.02591),(1.335,2.149,0.02606),(1.344,2.1503,0.02622),(1.353,2.1517,0.02638),(1.362,2.153,0.02654),(1.371,2.1544,0.0267),(1.38,2.1558,0.02686),(1.389,2.1572,0.02702),(1.398,2.1586,0.02719),(1.407,2.1601,0.02736),(1.416,2.1615,0.02752),(1.425,2.163,0.02769),(1.434,2.1644,0.02787),(1.443,2.1659,0.02804),(1.452,2.1674,0.02821),(1.461,2.1689,0.02839),(1.47,2.1704,0.02857),(1.479,2.172,0.02875),(1.488,2.1735,0.02893),(1.497,2.1751,0.02912),(1.506,2.1766,0.02931),(1.515,2.1782,0.02949),(1.524,2.1798,0.02969),(1.533,2.1815,0.02988),(1.542,2.1831,0.03007),(1.551,2.1848,0.03027),(1.56,2.1864,0.03047),(1.569,2.1881,0.03067),(1.578,2.1898,0.03088),(1.587,2.1915,0.03109),(1.596,2.1933,0.03129),(1.606,2.195,0.03151),(1.615,2.1968,0.03172),(1.624,2.1986,0.03194),(1.633,2.2004,0.03216),(1.642,2.2022,0.03238),(1.651,2.204,0.03261),(1.66,2.2059,0.03284),(1.669,2.2078,0.03307),(1.678,2.2097,0.03331),(1.687,2.2116,0.03355),(1.696,2.2136,0.03379),(1.705,2.2155,0.03404),(1.714,2.2175,0.03429),(1.723,2.2195,0.03454),(1.732,2.2215,0.0348),(1.741,2.2236,0.03506),(1.75,2.2257,0.03533),(1.759,2.2278,0.0356),(1.768,2.2299,0.03588),(1.777,2.2321,0.03616),(1.786,2.2342,0.03644),(1.795,2.2365,0.03673),(1.804,2.2387,0.03703),(1.813,2.2409,0.03733),(1.822,2.2432,0.03763),(1.831,2.2456,0.03795),(1.84,2.2479,0.03826),(1.849,2.2503,0.03859),(1.858,2.2527,0.03892),(1.867,2.2551,0.03926),(1.876,2.2576,0.03961),(1.885,2.2601,0.03996),(1.894,2.2627,0.04032),(1.903,2.2653,0.04069),(1.912,2.2679,0.04107),(1.921,2.2705,0.04146),(1.93,2.2733,0.04186),(1.939,2.276,0.04226),(1.948,2.2788,0.04268),(1.957,2.2816,0.04311),(1.966,2.2845,0.04356),(1.975,2.2874,0.04401),(1.984,2.2904,0.04448),(1.993,2.2934,0.04496),(2.002,2.2965,0.04546),(2.011,2.2996,0.04597),(2.02,2.3028,0.04651),(2.029,2.306,0.04705),(2.038,2.3093,0.04762),(2.047,2.3127,0.04821),(2.056,2.3162,0.04882),(2.065,2.3197,0.04946),(2.075,2.3232,0.05012),(2.084,2.3269,0.05081),(2.093,2.3306,0.05153),(2.102,2.3344,0.05228),(2.111,2.3383,0.05307),(2.12,2.3423,0.05389),(2.129,2.3464,0.05476),(2.138,2.3506,0.05567),(2.147,2.3549,0.05663),(2.156,2.3593,0.05765),(2.165,2.3638,0.05873),(2.174,2.3685,0.05987),(2.183,2.3733,0.06109),(2.192,2.3782,0.06239),(2.201,2.3833,0.06378),(2.21,2.3885,0.06527),(2.219,2.3939,0.06687),(2.228,2.3995,0.06861),(2.237,2.4053,0.0705),(2.246,2.4113,0.07255),(2.255,2.4175,0.07479),(2.264,2.4239,0.07726),(2.273,2.4307,0.07998),(2.282,2.4377,0.083),(2.291,2.445,0.08637),(2.3,2.4526,0.09015),(2.309,2.4606,0.09441),(2.318,2.4689,0.09926),(2.327,2.4777,0.10481),(2.336,2.4869,0.1112),(2.345,2.4966,0.11862),(2.354,2.5067,0.1273),(2.363,2.5173,0.13753),(2.372,2.5284,0.14966),(2.381,2.5399,0.16411),(2.39,2.5517,0.18136),(2.399,2.5634,0.20189),(2.408,2.5748,0.22607),(2.417,2.5851,0.25387),(2.426,2.5933,0.2845),(2.435,2.5982,0.31595),(2.444,2.5985,0.34492),(2.453,2.5932,0.36758),(2.462,2.5828,0.38118),(2.471,2.5691,0.38552),(2.48,2.5547,0.38305),(2.489,2.5408,0.37746),(2.498,2.5269,0.37186),(2.507,2.5112,0.3676),(2.516,2.4922,0.36426),(2.525,2.4705,0.36051),(2.535,2.4489,0.3553),(2.544,2.4307,0.34868),(2.553,2.4181,0.34158),(2.562,2.4115,0.33512),(2.571,2.4098,0.33006),(2.58,2.4114,0.32667),(2.589,2.4151,0.32492),(2.598,2.4197,0.3246),(2.607,2.4246,0.32545),(2.616,2.4294,0.32722),(2.625,2.4339,0.3297),(2.634,2.4381,0.33272),(2.643,2.4417,0.33613),(2.652,2.4449,0.33984),(2.661,2.4476,0.34375),(2.67,2.45,0.3478),(2.679,2.4519,0.35194),(2.688,2.4535,0.35613),(2.697,2.4548,0.36035),(2.706,2.4558,0.36456),(2.715,2.4565,0.36876),(2.724,2.457,0.37291),(2.733,2.4573,0.37703),(2.742,2.4574,0.38109),(2.751,2.4574,0.38509),(2.76,2.4572,0.38903),(2.769,2.4568,0.39289),(2.778,2.4564,0.39669),(2.787,2.4558,0.40042),(2.796,2.4552,0.40408),(2.805,2.4544,0.40766),(2.814,2.4536,0.41117),(2.823,2.4527,0.4146),(2.832,2.4518,0.41797),(2.841,2.4508,0.42126),(2.85,2.4497,0.42449),(2.859,2.4486,0.42764),(2.868,2.4475,0.43073),(2.877,2.4463,0.43375),(2.886,2.4452,0.43671),(2.895,2.444,0.4396),(2.904,2.4427,0.44243),(2.913,2.4415,0.44519),(2.922,2.4402,0.4479),(2.931,2.4389,0.45055),(2.94,2.4377,0.45314),(2.949,2.4364,0.45568),(2.958,2.4351,0.45816),(2.967,2.4338,0.46059),(2.976,2.4325,0.46297),(2.985,2.4312,0.4653),(2.994,2.4299,0.46757),(3.004,2.4287,0.4698),(3.013,2.4274,0.47199),(3.022,2.4261,0.47413),(3.031,2.4249,0.47622),(3.04,2.4236,0.47827),(3.049,2.4224,0.48028),(3.058,2.4211,0.48225),(3.067,2.4199,0.48418),(3.076,2.4187,0.48607),(3.085,2.4176,0.48792),(3.094,2.4164,0.48974),(3.103,2.4152,0.49152),(3.112,2.4141,0.49326),(3.121,2.413,0.49498),(3.13,2.4119,0.49666),(3.139,2.4108,0.49831),(3.148,2.4097,0.49992),(3.157,2.4087,0.50151),(3.166,2.4077,0.50307),(3.175,2.4067,0.5046),(3.184,2.4057,0.5061),(3.193,2.4047,0.50758),(3.202,2.4038,0.50903),(3.211,2.4029,0.51046),(3.22,2.402,0.51186),(3.229,2.4011,0.51324),(3.238,2.4002,0.5146),(3.247,2.3994,0.51594),(3.256,2.3986,0.51725),(3.265,2.3978,0.51855),(3.274,2.3971,0.51982),(3.283,2.3963,0.52108),(3.292,2.3956,0.52232),(3.301,2.3949,0.52354),(3.31,2.3943,0.52474),(3.319,2.3937,0.52593),(3.328,2.393,0.52711),(3.337,2.3925,0.52826),(3.346,2.3919,0.52941),(3.355,2.3914,0.53054),(3.364,2.3909,0.53166),(3.373,2.3904,0.53277),(3.382,2.3899,0.53387),(3.391,2.3895,0.53495),(3.4,2.3891,0.53603),(3.409,2.3887,0.5371),(3.418,2.3884,0.53816),(3.427,2.3881,0.53921),(3.436,2.3878,0.54025),(3.445,2.3875,0.54129),(3.454,2.3873,0.54232),(3.463,2.387,0.54334),(3.473,2.3869,0.54436),(3.481,2.3867,0.54538),(3.49,2.3866,0.54639),(3.5,2.3865,0.5474),(3.509,2.3864,0.54841),(3.518,2.3863,0.54942),(3.527,2.3863,0.55043),(3.536,2.3863,0.55143),(3.545,2.3864,0.55244),(3.554,2.3864,0.55345),(3.563,2.3865,0.55446),(3.572,2.3866,0.55547),(3.581,2.3868,0.55648),(3.59,2.387,0.5575),(3.599,2.3872,0.55853),(3.608,2.3874,0.55956),(3.617,2.3876,0.56059),(3.626,2.3879,0.56163),(3.635,2.3883,0.56268),(3.644,2.3886,0.56374),(3.653,2.389,0.56481),(3.662,2.3894,0.56588),(3.671,2.3898,0.56697),(3.68,2.3903,0.56807),(3.689,2.3908,0.56918),(3.698,2.3913,0.5703),(3.707,2.3918,0.57144),(3.716,2.3924,0.57259),(3.725,2.393,0.57375),(3.734,2.3937,0.57494),(3.743,2.3943,0.57614),(3.752,2.395,0.57736),(3.761,2.3957,0.57859),(3.77,2.3965,0.57985),(3.779,2.3973,0.58113),(3.788,2.3981,0.58243),(3.797,2.3989,0.58375),(3.806,2.3998,0.5851),(3.815,2.4007,0.58647),(3.824,2.4017,0.58786),(3.833,2.4026,0.58929),(3.842,2.4036,0.59074),(3.851,2.4046,0.59222),(3.86,2.4057,0.59374),(3.869,2.4068,0.59528),(3.878,2.4079,0.59686),(3.887,2.409,0.59847),(3.896,2.4102,0.60011),(3.905,2.4114,0.6018),(3.914,2.4126,0.60352),(3.923,2.4139,0.60528),(3.933,2.4152,0.60708),(3.942,2.4165,0.60893),(3.951,2.4178,0.61081),(3.96,2.4192,0.61275),(3.969,2.4206,0.61473),(3.978,2.4221,0.61676),(3.987,2.4235,0.61884),(3.996,2.425,0.62097),(4.005,2.4266,0.62316),(4.014,2.4281,0.6254),(4.023,2.4297,0.6277),(4.032,2.4313,0.63006),(4.041,2.433,0.63248),(4.05,2.4347,0.63496),(4.059,2.4364,0.63751),(4.068,2.4381,0.64013),(4.077,2.4399,0.64282),(4.086,2.4417,0.64558),(4.095,2.4435,0.64841),(4.104,2.4453,0.65132),(4.113,2.4472,0.65432),(4.122,2.4491,0.65739),(4.131,2.4511,0.66055),(4.14,2.4531,0.66379),(4.149,2.4551,0.66713),(4.158,2.4571,0.67055),(4.167,2.4591,0.67408),(4.176,2.4612,0.6777),(4.185,2.4633,0.68142),(4.194,2.4655,0.68525),(4.203,2.4676,0.68918),(4.212,2.4698,0.69323),(4.221,2.4721,0.69739),(4.23,2.4743,0.70167),(4.239,2.4766,0.70607),(4.248,2.4789,0.7106),(4.257,2.4812,0.71525),(4.266,2.4836,0.72004),(4.275,2.4859,0.72496),(4.284,2.4883,0.73002),(4.293,2.4908,0.73523),(4.302,2.4932,0.74058),(4.311,2.4957,0.74608),(4.32,2.4982,0.75174),(4.329,2.5007,0.75756),(4.338,2.5033,0.76354),(4.347,2.5058,0.76969),(4.356,2.5084,0.77601),(4.365,2.511,0.7825),(4.374,2.5137,0.78917),(4.383,2.5163,0.79603),(4.392,2.519,0.80307),(4.402,2.5217,0.8103),(4.41,2.5244,0.81773),(4.42,2.5271,0.82536),(4.429,2.5298,0.83318),(4.438,2.5326,0.84121),(4.446,2.5353,0.84944),(4.456,2.5381,0.85789),(4.465,2.5409,0.86654),(4.474,2.5437,0.87541),(4.483,2.5465,0.88449),(4.492,2.5494,0.89379),(4.501,2.5522,0.9033),(4.51,2.555,0.91303),(4.519,2.5579,0.92298),(4.528,2.5607,0.93314),(4.537,2.5636,0.94351),(4.546,2.5664,0.95409),(4.555,2.5693,0.96488),(4.564,2.5721,0.97587),(4.573,2.575,0.98705),(4.582,2.5778,0.99843),(4.591,2.5807,1.01),(4.6,2.5835,1.0217),(4.609,2.5863,1.0336),(4.618,2.5892,1.0456),(4.627,2.592,1.0578),(4.636,2.5947,1.0701),(4.645,2.5975,1.0826),(4.654,2.6002,1.0951),(4.663,2.603,1.1077),(4.672,2.6057,1.1203),(4.681,2.6083,1.133),(4.69,2.611,1.1457),(4.699,2.6136,1.1584),(4.708,2.6161,1.1711),(4.717,2.6187,1.1837),(4.726,2.6212,1.1962),(4.735,2.6236,1.2086),(4.744,2.626,1.2209),(4.753,2.6284,1.2331),(4.762,2.6307,1.245),(4.771,2.6329,1.2567),(4.78,2.6351,1.2682),(4.789,2.6372,1.2795),(4.798,2.6392,1.2904),(4.807,2.6412,1.301),(4.816,2.6431,1.3113),(4.825,2.6449,1.3213),(4.834,2.6466,1.3308),(4.843,2.6482,1.34),(4.852,2.6497,1.3488),(4.861,2.6512,1.3571),(4.87,2.6525,1.365),(4.879,2.6537,1.3725),(4.889,2.6548,1.3796),(4.898,2.6558,1.3862),(4.907,2.6566,1.3923),(4.916,2.6574,1.398),(4.925,2.658,1.4033),(4.934,2.6584,1.4082),(4.943,2.6587,1.4127),(4.952,2.6589,1.4168),(4.961,2.6588,1.4205),(4.97,2.6587,1.4238),(4.979,2.6583,1.4269),(4.988,2.6578,1.4296),(4.997,2.6571,1.432),(5.006,2.6562,1.4342),(5.015,2.6551,1.4362),(5.024,2.6538,1.4379),(5.033,2.6523,1.4395),(5.042,2.6506,1.441),(5.051,2.6487,1.4424),(5.06,2.6465,1.4436),(5.069,2.6441,1.4449),(5.078,2.6415,1.4461),(5.087,2.6386,1.4473),(5.096,2.6355,1.4486),(5.105,2.6322,1.45),(5.114,2.6285,1.4515),(5.123,2.6247,1.4531),(5.132,2.6205,1.4548),(5.141,2.6161,1.4568),(5.15,2.6114,1.4589),(5.159,2.6064,1.4612),(5.168,2.6011,1.4638),(5.177,2.5955,1.4667),(5.186,2.5897,1.4698),(5.195,2.5835,1.4732),(5.204,2.5771,1.4769),(5.213,2.5703,1.4809),(5.222,2.5633,1.4851),(5.231,2.556,1.4897),(5.24,2.5483,1.4946),(5.249,2.5403,1.4998),(5.258,2.5321,1.5053),(5.267,2.5235,1.511),(5.276,2.5147,1.5171),(5.285,2.5055,1.5233),(5.294,2.4961,1.5298),(5.303,2.4863,1.5366),(5.313,2.4763,1.5435),(5.321,2.466,1.5506),(5.331,2.4553,1.5578),(5.339,2.4445,1.5651),(5.348,2.4333,1.5724),(5.357,2.4219,1.5799),(5.367,2.4102,1.5873),(5.376,2.3983,1.5946),(5.384,2.3861,1.6019),(5.394,2.3737,1.6091),(5.403,2.3611,1.6161),(5.412,2.3482,1.6229),(5.421,2.3352,1.6295),(5.43,2.3219,1.6358),(5.439,2.3085,1.6418),(5.448,2.2949,1.6475),(5.457,2.2811,1.6528),(5.466,2.2671,1.6576),(5.475,2.253,1.6621),(5.484,2.2388,1.6661),(5.493,2.2244,1.6696),(5.502,2.21,1.6727),(5.511,2.1954,1.6752),(5.52,2.1808,1.6773),(5.529,2.1661,1.6788),(5.538,2.1513,1.6798),(5.547,2.1364,1.6803),(5.556,2.1216,1.6803),(5.565,2.1067,1.6798),(5.574,2.0917,1.6788),(5.583,2.0768,1.6773),(5.592,2.0619,1.6754),(5.601,2.047,1.673),(5.61,2.0321,1.6702),(5.619,2.0173,1.667),(5.628,2.0025,1.6635),(5.637,1.9877,1.6596),(5.646,1.9731,1.6555),(5.655,1.9585,1.651),(5.664,1.944,1.6463),(5.673,1.9296,1.6414),(5.682,1.9153,1.6362),(5.691,1.9012,1.6309),(5.7,1.8871,1.6255)])

### Fitting at 250~700nm

In [ ]:
fitData('Ag', [250, 750], [1, 3], [9, 5, 4])

In [ ]:
fitData('Au', [250, 750], [1, 3], [9, 5, 3, 2])

In [ ]:
fitData('Cu', [250, 750], [1, 3], [9, 5, 3, 2], True)

In [ ]:
fitData('Al', [250, 750], [1, 3], [9])

In [ ]:
fitData('Cr', [250, 750], [1, 3], [9])

In [ ]:
fitData('Ni', [250, 750], [1, 3], [8, 4, [1, 0, 9, 9]])

In [ ]:
fitData('Ti', [250, 750], [1, 3], [8, 4, 3, 2], True)

In [ ]:
fitData('Pd', [250, 750], [1, 3], [10, 4, 3, 2])

In [ ]:
fitData('Pt', [250, 750], [1, 1], [9, [1, 1, 5, 14]], True)

In [ ]:
fitData('Ge', [250, 750], [0, 5], [4, 4, 4, 3, 3], True)

In [ ]:
fitData('Si', [250, 750], [0, 4], [4, 4, 4, 4], True)

In [ ]:
fitData('aSi', [250, 750], [0, 4], [4, 4, 4, 4], True)

### Fitting at 400~2000nm

In [ ]:
fitData('Ag_JC', [400, 2000], [1, 0])

In [ ]:
fitData('Au', [400, 2000], [1, 2], [9, 3, 3])

In [ ]:
fitData('Cu', [400, 2000], [1, 2], [9, 5], True)

In [ ]:
fitData('Al', [400, 2000], [1, 3], [9], True)

In [ ]:
fitData('Cr', [400, 2000], [1, 3], [5, 3])

In [ ]:
fitData('Ni', [400, 2000], [1, 2], [5, [1, 1, 9, 8]])

In [ ]:
fitData('Ti', [400, 2000], [1, 3], [8, 2, 2, 2], True)

In [ ]:
fitData('Pd', [400, 2000], [1, 3], [7])

In [ ]:
fitData('Pt', [400, 2000], [1, 4], [7, [1, 1, 9, 9]], True)

In [ ]:
fitData('Ge', [400, 2000], [0, 3])

In [ ]:
fitData('Si', [400, 2000], [0, 1])

In [ ]:
fitData('aSi', [400, 2000], [0, 2], [3, 2], True)

### Fitting at 400~1040nm

In [ ]:
fitData('oCdS', [400, 1040], [0, 4], [5, 3, 3, 2], True)

In [ ]:
fitData('eCdS', [400, 1040], [0, 4], [5, 3, 3, 2], True)

### All fitting results

In [33]:
for region, DB in ParameterDB.items():
    print(f'/* {region}nm */')
    for matter, result in DB.items():
        print(f'matter {matter} = {result};')

/* 250-750nm */
matter Ag = {{1.9201},{{9.3937,0.0302},{4.2333,1.2373,1.3399,1.6141},{3.9548,0.259,-0.1356,0.2961},{0.8394,0.5873,2.652,6.1529}}};
matter Au = {{2.03},{{8.4306,0.0},{4.3848,1.2748,-0.2723,-3.7849},{4.7338,2.0464,5.3396,4.7241},{2.6147,0.3187,0.371,0.723}}};
matter Cu = {{1.7411},{{8.8195,0.0835},{4.6887,0.7044,0.3576,0.4646},{3.4555,2.8361,7.7571,-0.1944},{2.1607,0.2625,0.0964,1.0257}}};
matter Al = {{1.0},{{13.4862,0.1406},{2.24,1.4861,3.8125,-0.5311},{1.8272,0.3877,3.8946,2.3493},{1.5717,0.2272,9.7451,-1.7878}}};
matter Cr = {{1.0009},{{9.1925,0.7681},{3.4893,0.4922,0.7387,-0.0827},{0.9432,0.2803,-21.3216,29.9879},{2.4741,0.6165,2.6444,-5.3032}}};
matter Ni = {{2.04},{{7.8334,0.5169},{4.3024,1.0858,2.3911,1.602},{1.3054,0.4506,8.9446,6.5275},{3.3426,1.7382,1.3531,-3.329}}};
matter Ti = {{1.0},{{12.3768,5.5499},{3.9759,0.8038,-1.0395,-0.3514},{2.1995,1.048,-3.7293,0.9582},{1.0376,0.2488,-0.596,6.2558}}};
matter Pd = {{1.7139},{{9.3495,0.7156},{4.7399,0.7234,0.2268,0.06